In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import os
from os import walk
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

os.getcwd()

DATA_PATH = 'C:\\AI Learning\\Hacker Earth Projects\\Employee Challenge\\dataset'

train_data = pd.read_csv(os.path.join(DATA_PATH,"train.csv"))
test_data = pd.read_csv(os.path.join(DATA_PATH,"test.csv"))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 24 columns):
Employee_ID                  7000 non-null object
Gender                       7000 non-null object
Age                          6588 non-null float64
Education_Level              7000 non-null int64
Relationship_Status          7000 non-null object
Hometown                     7000 non-null object
Unit                         7000 non-null object
Decision_skill_possess       7000 non-null object
Time_of_service              6856 non-null float64
Time_since_promotion         7000 non-null int64
growth_rate                  7000 non-null int64
Travel_Rate                  7000 non-null int64
Post_Level                   7000 non-null int64
Pay_Scale                    6991 non-null float64
Compensation_and_Benefits    7000 non-null object
Work_Life_balance            6989 non-null float64
VAR1                         7000 non-null int64
VAR2                         6423 non-null fl

In [25]:
for col in train_data.columns.tolist():
    print("data for ", col)
    print(train_data[col].value_counts())
#train_data.hist()
#corr_matrix = train_data.corr()
#corr_matrix

#train_data.plot(kind="scatter", x="Time_of_service", y="Attrition_rate")

#Variables to classify into buckets - Time_since_promotion, Pay_Scale (only 1,2,3) , Compensation_and_Benefits (0,1), 

data for  Employee_ID
EID_6319     1
EID_8773     1
EID_9006     1
EID_4376     1
EID_21644    1
EID_4129     1
EID_7146     1
EID_21451    1
EID_15999    1
EID_23111    1
EID_3067     1
EID_19215    1
EID_11699    1
EID_21345    1
EID_22764    1
EID_5062     1
EID_8626     1
EID_1966     1
EID_17365    1
EID_2765     1
EID_4165     1
EID_2302     1
EID_11377    1
EID_25807    1
EID_18970    1
EID_8237     1
EID_24050    1
EID_17987    1
EID_4569     1
EID_17559    1
            ..
EID_12360    1
EID_7396     1
EID_19088    1
EID_2351     1
EID_2393     1
EID_12757    1
EID_7320     1
EID_10638    1
EID_9753     1
EID_14607    1
EID_5253     1
EID_11327    1
EID_6733     1
EID_2380     1
EID_4931     1
EID_6290     1
EID_1251     1
EID_19921    1
EID_11525    1
EID_25908    1
EID_2294     1
EID_9028     1
EID_7110     1
EID_10308    1
EID_11743    1
EID_7922     1
EID_19075    1
EID_12423    1
EID_22510    1
EID_25244    1
Name: Employee_ID, Length: 7000, dtype: int64
data for  Gender


In [3]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [4]:
class MyLabelEncoder(TransformerMixin):

    def __init__(self, *args, **kwargs):

        self.encoder = LabelEncoder(*args, **kwargs)

    def fit(self, x, y=0):

        self.encoder.fit(x)

        return self

    def transform(self, x, y=0):

        return self.encoder.transform(x)

In [5]:
#train_data.columns.tolist()

num_attribs = ['Age',  'Education_Level', 'Time_of_service', 'Time_since_promotion', 'growth_rate', 'Travel_Rate', 'Post_Level', 'Pay_Scale',  'Work_Life_balance', 'VAR1','VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6', 'VAR7']
cat_attribs = ['Gender', 'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess', 'Compensation_and_Benefits' ]

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import FeatureUnion


num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', OneHotEncoder(sparse=False, handle_unknown ='ignore')),
    ]) #('label_encoder', MyLabelEncoder()),

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])


In [7]:
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
#data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
#values = np.array(data)
#print(values)

#label_encoder = LabelEncoder()
#integer_encoded = label_encoder.fit_transform(values)
#print(integer_encoded)

#print(train_data[cat_attribs].shape)

#onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#onehot_encoded = onehot_encoder.fit_transform(train_data[cat_attribs])  #values.reshape(len(values), 1))#integer_encoded
#print(onehot_encoded)
#print(onehot_encoded.shape)


yActual = train_data["Attrition_rate"]
#finalData = cat_pipeline.fit_transform(train_data.drop(["Employee_ID", "Attrition_rate"], axis = 1))
finalData = full_pipeline.fit_transform(train_data.drop(["Employee_ID", "Attrition_rate"], axis = 1))
print(finalData.shape)

(7000, 46)


In [8]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)

#logreg = LogisticRegression()
#logreg.fit(finalData, yActual)
forest_reg.fit(finalData, yActual)

C:\Users\Gandharv\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [9]:
y_pred = forest_reg.predict(finalData)
forest_mse = mean_squared_error(yActual, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse


0.08422123137535636

In [14]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(learning_rate=0.02, booster='gbtree', n_estimators=600, objective='reg:linear', random_state=42)

param_grid = [
    {"max_depth":[2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
    "min_child_weight": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
    "subsample": [i/10 for i in range(2,11)],
    "colsample" : [i/10 for i in range(2,11)]}
]

final_xgb_model= GridSearchCV(xgb_model, param_grid, cv = 10, scoring='neg_mean_squared_error', n_jobs=5)

final_xgb_model.fit(finalData, yActual)

KeyboardInterrupt: 

In [76]:
param_grid = [
    {'bootstrap': [True, False], 'n_estimators': [3, 10, 50, 100, 200], 'max_features': [2, 3, 4]},
]

grid_search = GridSearchCV(forest_reg, param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs = 4)

# Fit on data

grid_search.fit(finalData, yActual)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=4,
             param_grid=[{'bootstrap': [True, False], 'max_featu

In [77]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.21641148797018478 {'bootstrap': True, 'max_features': 2, 'n_estimators': 3}
0.19586655419762183 {'bootstrap': True, 'max_features': 2, 'n_estimators': 10}
0.18795343864449454 {'bootstrap': True, 'max_features': 2, 'n_estimators': 50}
0.18734448517475338 {'bootstrap': True, 'max_features': 2, 'n_estimators': 100}
0.18685632008149414 {'bootstrap': True, 'max_features': 2, 'n_estimators': 200}
0.21421513728444302 {'bootstrap': True, 'max_features': 3, 'n_estimators': 3}
0.19552146809390702 {'bootstrap': True, 'max_features': 3, 'n_estimators': 10}
0.18880108442644872 {'bootstrap': True, 'max_features': 3, 'n_estimators': 50}
0.18754369490738496 {'bootstrap': True, 'max_features': 3, 'n_estimators': 100}
0.18722303433240783 {'bootstrap': True, 'max_features': 3, 'n_estimators': 200}
0.21572135221982372 {'bootstrap': True, 'max_features': 4, 'n_estimators': 3}
0.19636440044589587 {'bootstrap': True, 'max_features': 4, 'n_estimators': 10}
0.18868338824180694 {'bootstrap': True, 'max_featur

In [80]:
y_pred = grid_search.predict(finalData)
grid_search_mse = mean_squared_error(yActual, y_pred)
grid_search_mse = np.sqrt(grid_search_mse)
grid_search_mse


#f1_score(yActual, y_pred, average = 'weighted')
#recall_score(yActual, y_pred, average='weighted')

0.06944331033939499

In [83]:
finalTestData = full_pipeline.fit_transform(test_data.drop(["Employee_ID"], axis = 1))
test_pred = grid_search.predict(finalTestData)


In [84]:
test_data["Attrition_rate"] = test_pred
test_data


,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_22713,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,...,type2,1.0,3,-0.9612,-0.4537,2.0,1,8,4,0.200614
1,EID_9658,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,...,type2,1.0,4,-0.9612,0.7075,1.0,2,8,2,0.225615
2,EID_22203,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,...,type3,1.0,4,-0.1048,0.7075,2.0,1,9,3,0.170081
3,EID_7652,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,...,type0,4.0,3,-0.1048,0.7075,2.0,2,8,3,0.200159
4,EID_6516,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,...,type2,4.0,4,1.6081,0.7075,2.0,2,7,4,0.207707
5,EID_20283,F,22.0,4,Married,Franklin,IT,Behavioral,3.0,1,...,type2,1.0,3,NaN,-0.4537,2.0,3,8,3,0.176771
6,EID_21014,M,42.0,3,Married,Washington,Purchasing,Analytical,6.0,4,...,type2,1.0,3,0.7516,1.8688,2.0,3,7,3,0.190887
7,EID_7693,F,41.0,2,Married,Springfield,Sales,Conceptual,4.0,4,...,type2,1.0,3,-0.1048,1.8688,2.0,2,7,5,0.186125
8,EID_13232,M,31.0,1,Single,Springfield,IT,Analytical,7.0,3,...,type2,3.0,3,1.6081,0.7075,2.0,2,7,4,0.171115
9,EID_6515,M,48.0,2,Single,Springfield,R&D,Conceptual,16.0,1,...,type0,3.0,3,-0.1048,-0.4537,2.0,4,7,4,0.226749


In [85]:
final_submission = test_data[["Employee_ID", "Attrition_rate"]]

In [86]:
final_submission

,Employee_ID,Attrition_rate
0,EID_22713,0.200614
1,EID_9658,0.225615
2,EID_22203,0.170081
3,EID_7652,0.200159
4,EID_6516,0.207707
5,EID_20283,0.176771
6,EID_21014,0.190887
7,EID_7693,0.186125
8,EID_13232,0.171115
9,EID_6515,0.226749


In [87]:
final_submission.to_csv(os.path.join(DATA_PATH,"finalSubmission.csv"))